# Classify_Face Notebook

In [1]:
# This Notebook is used to test the Neural network using a webcam and the previously generated model
import numpy as np
import cv2
from keras.models import load_model
from Preprocessing.DCT import DCT2D

Using TensorFlow backend.


In [2]:
# Load in projection matrices 
X = np.load("./PP_Data/X.npy")
Z = np.load("./PP_Data/Z.npy")

In [3]:
CLASSES = ["Connor", "Dad", "Maddie", "Mom"] # Create classification labels based on categorical labels
model = load_model("./Models/model.hdf5") # load previously generated model

In [4]:
# This function is used to recognize faces using the variables loaded above
# Since the function would require a larger number of parameters I chose to not place it into its own python package
# like for PCA2D, DCT2D, etc.
# The method for getting a prediction and placing labels is based on 
# https://www.pyimagesearch.com/2016/09/26/a-simple-neural-network-with-python-and-keras/

def recognize_faces():
    
    # Load the haarcascade provided by OpenCV for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")    
    
    webcam = cv2.VideoCapture(0)
    
    success = webcam.isOpened()
    if success == False:
        print('Error: Camera could not be opened')
    else:
        print('Success: Grabbed the camera')
    
    p = 0 # padding for cropped image if necessary
    
    while True:
        ret, frame = webcam.read()
        frame = cv2.flip(frame, 1) # Otherwise video is not mirrored
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Get grayscale frame
        
        # Detect faces in current frame
        faces = face_cascade.detectMultiScale(frame, scaleFactor=1.5, minNeighbors=5)
        if ret: # ensure frame is captured
            for (x,y,w,h) in faces:
                # Loop through all faces in frame and perform classification
                # Place rectangle around detected faces 
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2);
                
                # Get cropped face frame
                cropped_gray = gray[y-p+1:y+h+p, x-p+1:x+w+p]
                # Perform preprocessing similar to what was done for the training set
                # Ensure scaling matches scaling done in Preprocessing step
                scaled = cv2.resize(cropped_gray, (300,300), interpolation=cv2.INTER_LINEAR)
                norm_img = np.zeros((300,300))
                norm_img = cv2.normalize(scaled, norm_img, 0, 255, cv2.NORM_MINMAX)
                dct_image = np.uint8(DCT2D(norm_img).dct_compression())
                # Use projection matrices from the training data
                # to transform captured face into principle component space of the training images
                face = Z.T @ dct_image @ X
                # flatten PC space into feature vector
                features = np.array([face.flatten()])
                
                # use model to make prediction and capture highest probable result
                probs = model.predict(features)[0]
                prediction = probs.argmax(axis=0)

                
                # If the prediction has a high enough accuracy based on the model
                # display the corresponding label on top of the face being detected
                label = ""
                if probs[prediction] > .60:
                    label = "{}: {:.2f}%".format(CLASSES[prediction],
                    probs[prediction] * 100)

                    cv2.putText(frame, label, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                                1.0, (0, 255, 0), 3, cv2.LINE_AA)

            cv2.imshow("image", frame)
        
        # Monitor keystrokes
        k = cv2.waitKey(1)

        if k & 0xFF == ord('q'):
            # Press q to quit
            print("Quitting...")
            break
       
    webcam.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [5]:
recognize_faces()

Success: Grabbed the camera
Quitting...
